In [1]:
!pip install pymupdf
!pip install faiss-cpu
!pip install sentence_transformers
!pip install Pillow
!pip install accelerate
!pip install --upgrade mistral_common
!pip install --upgrade vllm

  Using cached sentence_transformers-4.1.0-py3-none-any.whl.metadata (13 kB)
  Using cached transformers-4.51.3-py3-none-any.whl.metadata (38 kB)
  Using cached torch-2.7.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (29 kB)
  Using cached huggingface_hub-0.30.2-py3-none-any.whl.metadata (13 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.6.77-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.6.77-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.6.80-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.5.1.17-py3-none-manylinux_2_28_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.6.4.1-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nv

# Nouvelle section

In [2]:
import os
import fitz  # PyMuPDF pour l'extraction de texte et d'images
import numpy as np
import faiss
import matplotlib.pyplot as plt
import pandas as pd
from nltk.tokenize import sent_tokenize
# Import pour le NLP
from sentence_transformers import SentenceTransformer, CrossEncoder,util
from transformers import pipeline, BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import torch
import re
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch


/opt/conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_id = "mistralai/Mistral-7B-Instruct-v0.2"

# Chargement du modèle et du tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")

# Important : définir le pad_token
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading checkpoint shards: 100%|██████████| 3/3 [00:08<00:00,  2.97s/it]


In [19]:
def extract_text_from_pdf(pdf_path):
    """
    Extrait le texte brut d’un fichier PDF.

    Args:
        pdf_path (str): Chemin vers le fichier PDF.

    Returns:
        str: Texte complet extrait du PDF, concaténé page par page avec des sauts de ligne.
    """
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text() + "\n"
    return text


def chunk_text(text):
    """
    Découpe le texte en morceaux (chunks) de taille fixe avec recouvrement (overlap).

    Args:
        text (str): Texte complet à découper.

    Returns:
        list: Liste de chaînes de caractères, chaque chunk contenant environ 300 mots
              avec 30 mots de chevauchement pour préserver le contexte.
    """
    chunk_size, overlap = 300, 30
    words = text.split()
    chunks = []
    step = chunk_size - overlap
    for i in range(0, len(words), step):
        chunk = words[i:i+chunk_size]
        chunks.append(" ".join(chunk))
    return chunks


def encode_passages(passages, model):
    """
    Encode chaque passage textuel en vecteur d’embedding avec un modèle de type SentenceTransformer.

    Args:
        passages (list): Liste de chaînes de caractères à encoder.
        model (SentenceTransformer): Modèle d'embedding, par exemple E5 ou MiniLM.

    Returns:
        np.ndarray: Matrice d’embeddings de type float32 (shape: [n_passages, embedding_dim]).
    """
    embeddings = model.encode(passages, show_progress_bar=True)
    return np.array(embeddings, dtype=np.float32)


def build_faiss_index(embeddings):
    """
    Construit un index FAISS à partir des embeddings (utilise la distance L2).

    Args:
        embeddings (np.ndarray): Matrice des embeddings à indexer (shape: [n, d]).

    Returns:
        faiss.IndexFlatL2: Index FAISS entraîné et prêt pour les requêtes.
    """
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)
    return index


def retrieve_passages(query, passages, embedding_model, faiss_index, k=10):
    """
    Recherche les k passages les plus pertinents pour une requête donnée à l'aide de FAISS.

    Args:
        query (str): Question ou phrase à rechercher.
        passages (list): Liste des passages (chunks) d’origine.
        embedding_model (SentenceTransformer): Modèle d’embedding utilisé pour encoder la requête.
        faiss_index (faiss.IndexFlatL2): Index FAISS contenant les embeddings des passages.
        k (int): Nombre de résultats à retourner.

    Returns:
        list: Liste des k passages les plus proches (par similarité).
    """
    query_embedding = embedding_model.encode([query])
    distances, indices = faiss_index.search(np.array(query_embedding, dtype=np.float32), k)
    retrieved = [passages[idx] for idx in indices[0] if idx < len(passages)]
    return retrieved


def rerank_passages(query, passages, reranker, threshold=0.4):
    """
    Recalcule la pertinence des passages récupérés avec un modèle Cross-Encoder type MiniLM,
    puis filtre et trie les passages les plus pertinents.

    Args:
        query (str): Question initiale de l’utilisateur.
        passages (list): Passages récupérés initialement via FAISS.
        reranker (CrossEncoder): Modèle Cross-Encoder (type ms-marco) pour scoring binaire.
        threshold (float): Seuil minimal pour conserver un passage.

    Returns:
        list of tuples: Liste de (passage, score) triés par score décroissant.
    """
    pairs = [(query, passage) for passage in passages]
    scores = reranker.predict(pairs)
    filtered = [(passage, score) for passage, score in zip(passages, scores) if score > threshold]
    ranked = sorted(filtered, key=lambda x: x[1], reverse=True)
    return ranked

def generate_answer(query, ranked_passages, max_new_tokens=400):
    """
    Génère une réponse fiable à partir des passages, avec fallback si aucun passage pertinent.
    
    Args:
        query (str): Question posée
        ranked_passages (list): Liste des passages (textes)
        max_length (int): Longueur max de la réponse
        
    Returns:
        str: Réponse 
    """
    # 1. Créer le contexte formaté
    context = "\n".join([f"[Source {i+1}] {text}" for i, text in enumerate(ranked_passages)]) if ranked_passages else "Aucune source disponible"
    
    # 2. Prompt robuste avec instructions claires
    prompt = f"""<s>[INST]
    You are an expert in document analysis. Answer the question {query} using ONLY the sources provided.
    Structure your answer in a maximum of 3 sentences.

    Strict rules:
    1. If the sources do NOT contain the answer, say exactly: "Je n'ai pas trouvé d'information pertinente dans les documents."
    2. Be concise

    Question: {query}
    Sources: {context}
    Réponse : [/INST]"""

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.4,
        top_p=0.85,
        top_k=30,
        pad_token_id=tokenizer.pad_token_id
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return generated_text.split("[/INST]")[-1].strip()
    
# def generate_answer(query, ranked_passages, max_length=500):
#     """
#     Génère une réponse à une question à partir des passages fournis.
#     Accepte des passages avec ou sans scores.

#     Args:
#         query (str): La question posée.
#         ranked_passages (list): Liste de strings OU de tuples (passage, score).
#         max_length (int): Longueur maximale de la réponse (en tokens).

#     Returns:
#         str: Réponse générée par le modèle.
#     """

#     # Vérifie si chaque passage est un tuple (passage, score) ou juste une string
#     context = "\n".join([
#         f"[Source {i+1}] {p if isinstance(p, str) else p[0]}"
#         for i, p in enumerate(ranked_passages)
#     ])

#     prompt = f"""<s>[INST]
# You are an expert in sustainability and corporate responsibility.
# Answer the following question ONLY based on the sources provided.
# Structure the response as a single paragraph of less than 400 words with key points.
# If the sources do not contain enough information, reply: "The information provided does not allow to answer the question."

# Question: {query}

# Sources: {context}
# [/INST]"""

#     response = generator(
#         prompt,
#         max_new_tokens=max_length,
#         do_sample=True,
#         temperature=0.3,
#         top_k=30,
#         top_p=0.85,
#         num_return_sequences=1
#     )

#     generated_text = response[0]['generated_text']
#     return generated_text.split("[/INST]")[-1].strip()



def find_top_chunks_for_phrase(phrase, chunks, top_k, threshold):
    """
    Trouve les top_k chunks les plus similaires à une phrase pertinente.

    Args:
        phrase (str): La phrase de référence.
        chunks (List[str]): Tous les chunks du document.
        top_k (int): Nombre de passages à retourner.
        threshold (float): Score de similarité minimale.

    Returns:
        List[Tuple[int, float, str]]: (chunk_id, score, texte) triés par pertinence.
    """
    # Encodage
    phrase_embedding = embedding_model.encode(f"query: {phrase}", convert_to_tensor=True)
    chunk_texts = [f"passage: {chunk}" for chunk in chunks]
    chunk_embeddings = embedding_model.encode(chunk_texts, convert_to_tensor=True)

    # Similarité cosinus
    similarities = util.cos_sim(phrase_embedding, chunk_embeddings)[0]

    # Tri décroissant
    top_indices = torch.argsort(similarities, descending=True)

    # Récupération des top chunks au-dessus du seuil
    top_matches = []
    for idx in top_indices[:top_k * 2]:  # on élargit au cas où certains soient sous le seuil
        score = similarities[idx].item()
        if score >= threshold:
            top_matches.append((idx.item(), score, chunks[idx.item()]))
        if len(top_matches) == top_k:
            break

    return top_matches

def calculate_mrr(questions_data, retrieved_passages_by_question, threshold=0.85):
    """
    Calcule le MRR en comparant uniquement le premier passage du ground truth
    avec les chunks retrouvés (et non tous les chunks du corpus !).

    Args:
        questions_data (list): liste de questions + ground truth
            [
                {
                    "question": "...",
                    "relevant_chunks": ["..."]  # on prend le premier
                },
                ...
            ]
        retrieved_passages_by_question (dict): {question: [retrieved_chunks (texte)]}
        threshold (float): seuil de similarité cosinus pour considérer un match

    Returns:
        float: MRR global
    """
    reciprocal_ranks = []

    for q_data in questions_data:
        question = q_data["question"]
        ground_truth_chunk = q_data["relevant_chunks"][0]  # on compare au 1er chunk GT

        # Embedding du ground truth
        gt_embedding = model.encode(f"passage: {ground_truth_chunk}", convert_to_tensor=True)

        # Passages récupérés uniquement pour cette question
        retrieved_passages = retrieved_passages_by_question.get(question, [])

        found = False

        for rank, passage in enumerate(retrieved_passages, start=1):
            passage_embedding = model.encode(f"passage: {passage}", convert_to_tensor=True)
            sim = util.cos_sim(gt_embedding, passage_embedding)[0][0].item()

            if sim >= threshold:
                reciprocal_ranks.append(1 / rank)
                found = True
                break  # stop at first match

        if not found:
            reciprocal_ranks.append(0.0)

    return sum(reciprocal_ranks) / len(reciprocal_ranks) if reciprocal_ranks else 0.0

In [7]:
pdf_file = os.listdir("data")

# Nouvelle section

In [6]:
pdf_file

['20240903-bnpparibas-csr-presentation-2024.pdf',
 'bnp_paribas_2023_climate_report.pdf',
 'What-contribution-do-banks-make-to-the-ecological-transition.pdf',
 'climate-change-and-the-banking-industry.pdf',
 'as_101_climate_risk_banks_en.pdf',
 'ssm.202011finalguideonclimate-relatedandenvironmentalrisks~58213f6564.fr.pdf',
 'eib-group-2022-climate-bank-roadmap-progress-report.pdf']

In [8]:
full_text=""
for doc in pdf_file:
    pdf_path="data/"+doc
    full_text+=" "+ extract_text_from_pdf(pdf_path)

In [30]:
# taille avant ajout des images
len(full_text)

752720

In [21]:
full_caption

0      a black and white image of a black and white i...
1      a graph shows the percentage of the number of ...
2                 a green background with a white border
3         a black background with a white and red flower
4                             rest the world deved ratio
                             ...                        
330       a black background with a white and red flower
331                         the logo for the color wheel
332    a black and white image of a black and white i...
333    the sun shines over the mountains in meteor me...
334    a diagram showing the benefits of a green ener...
Name: caption, Length: 335, dtype: object

importation des description images

In [9]:
data=pd.read_csv("caption_image.csv")

In [10]:
full_caption=data["caption"]
len(full_caption.to_list())

335

### Initialisation du model

In [11]:
#  Extraction du texte et découpage en passages
passages = chunk_text(full_text)
# fusion avec la listes des captions
passages = passages + full_caption.to_list()
print(f"Nombre de passages extraits: {len(passages)}")

#  Chargement du embeddings e5
embedding_model = SentenceTransformer("intfloat/multilingual-e5-large")
passage_embeddings = encode_passages(passages, embedding_model)

faiss_index = build_faiss_index(passage_embeddings)

Nombre de passages extraits: 743


Batches: 100%|██████████| 24/24 [01:59<00:00,  4.97s/it]


In [23]:
if __name__ == "__main__":

    # Définir une requête utilisateur
    # query = "What is BNP Paribas' 2050 climate objective?"
    query = "Which graph shows the distribution of the number of customers by region?"
    #  Recherche initiale via FAISS
    # retrieved_passages = retrieve_passages(query, passages, embedding_model, faiss_index, k=5)
    retrieved_passages = find_top_chunks_for_phrase(query, passages, top_k=5, threshold=0.5)
    print("\nPassages récupérés (avant reranking) :")
    for p in retrieved_passages:
        print(" -", p[:100], "...")

    reponse = generate_answer(query, retrieved_passages)
    
    print("Question:", query)
    print("Réponse:", reponse)

    print("\n Ta rag RAG exécuté avec succès.")


Passages récupérés (avant reranking) :
 - (409, 0.7830719351768494, 'a graph shows the percentage of the number of people who are using the internet') ...
 - (448, 0.7825443744659424, 'a graph shows the number of people who are using the internet') ...
 - (437, 0.7818568348884583, 'a graph showing the number of different types of the global economy') ...
 - (705, 0.7785378098487854, 'a graph showing the number of different types of the internet') ...
 - (656, 0.7779329419136047, 'a graph with the number of the number of the number of the number of the number of the number of') ...
Question: Which graph shows the distribution of the number of customers by region?
Réponse: Je n'ai pas trouvé d'information pertinente dans les documents sur un graphique montrant la distribution du nombre de clients par région. Les sources fournies ne contiennent qu'des informations sur le nombre de personnes qui utilisent l'internet ou des types différents d'économie et d'internet.

 Ta rag RAG exécuté av

''

In [ ]:
#  Le vrai ground truth (phrases pertinentes extraites manuellement)
questions_data = [
    {
        "question": "Quel est l’objectif de BNP Paribas pour les prêts durables d’ici 2025 ?",
        "relevant_phrases": [
            "BNP Paribas vise un montant de prêts durables de 150 milliards d’euros d’ici 2025 (contre 117 milliards en 2023).",
            "Le groupe a réduit de 70% ses financements aux énergies fossiles depuis 2020.",
            "BNP Paribas est classé #1 mondial en finance durable en 2023."
        ]
    }
]

retrieved_ids = [passages.index(p) for p in retrieved_passages]

# 3. Structure pour le calcul MRR
retrieved_chunks_by_question = {
    "Quel est l’objectif de BNP Paribas pour les prêts durables d’ici 2025 ?": retrieved_ids
}

# Calcul du MRR
mrr = calculate_mrr(questions_data,all_chunks=passages, retrieved_chunks_by_question=retrieved_chunks_by_question, threshold=0.85
)

print(f" MRR: {mrr:.3f}")


### Great Truth pour dautre test mrr

##### On remplace query par la question et cesT OK

In [ ]:
questions_data=[ {
        "question": "Quels sont les engagements climatiques de BNP Paribas pour 2050 ?",
        "relevant_phrases": [
            "BNP Paribas s’est engagée à atteindre la neutralité carbone d’ici 2050.",
            "La banque aligne ses portefeuilles sur les scénarios de l’AIE."
        ]
    }
               ]

questions_data=[  {
        "question": "How does the EIB support adaptation to climate change in the European Union and beyond?",
        "relevant_phrases": [
            "In 2022 the EIB lent €1.8 billion for climate change adaptation, of which nearly 80% was in the European Union.",
            "Project examples from 2022 include: EIB support to investments in the water and wastewater infrastructure of the city of Warsaw [...] and support to Andalusia’s rural development programme to improve water catchment, prevent soil erosion.",
       "Beyond the European Union, EIB Global financed the Aqaba-Amman Water Desalination and Conveyance Project, the largest ever investment project for adapting the water sector to the impacts of climate change in Jordan."
        ]
    }
               ]


### Extraction image

In [ ]:

def extract_images_from_pdf(pdf_path, output_folder="images_extraits"):
    """
    Extrait toutes les images d'un fichier PDF et les enregistre dans un dossier "images_extraits".

    Args:
        pdf_path (str): Chemin vers le fichier PDF.
        output_folder (str): Dossier de sortie pour enregistrer les images.

    Returns:
        List[str]: Liste des chemins des images extraites.
    """
    doc = fitz.open(pdf_path)

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    image_paths = []
    image_count = 0

    for page_number in range(len(doc)):
        page = doc[page_number]
        images = page.get_images(full=True)

        for img_index, img in enumerate(images):
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            image_ext = base_image["ext"]
            pdf_name = os.path.splitext(os.path.basename(pdf_path))[0]
            image_filename = f"{output_folder}/{pdf_name}_page{page_number + 1}_img{img_index + 1}.{image_ext}"
            with open(image_filename, "wb") as f:
                f.write(image_bytes)

            image_paths.append(image_filename)
            image_count += 1

    print(f"{image_count} image(s) extraite(s) dans le dossier '{output_folder}'.")
    return image_paths


### image pertinente

#### Transcription img

In [6]:
import requests
import json
from time import sleep

In [ ]:
# from huggingface_hub import login

In [ ]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [4]:
from vllm import LLM
from vllm.sampling_params import SamplingParams

In [ ]:
model_name = "mistralai/Pixtral-12B-2409"
max_img_per_msg = 5

sampling_params = SamplingParams(max_tokens=8192, temperature=0.7)
# Lower max_num_seqs or max_model_len on low-VRAM GPUs.
llm = LLM(model=model_name, tokenizer_mode="mistral", limit_mm_per_prompt={"image": max_img_per_msg}, max_model_len=32768)



RuntimeError: Failed to infer device type, please set the environment variable `VLLM_LOGGING_LEVEL=DEBUG` to turn on verbose logging to help debug the issue.

In [ ]:
prompt = "Describe the following image."

url_1 = "https://huggingface.co/datasets/patrickvonplaten/random_img/resolve/main/yosemite.png"
url_2 = "https://picsum.photos/seed/picsum/200/300"
url_3 = "https://picsum.photos/id/32/512/512"

messages = [
    {
        "role": "user",
        "content": [{"type": "text", "text": prompt}, {"type": "image_url", "image_url": {"url": url_1}}, {"type": "image_url", "image_url": {"url": url_2}}],
    },
    {
        "role": "assistant",
        "content": "The images shows nature.",
    },
    {
        "role": "user",
        "content": "More details please and answer only in French!."
    },
    {
        "role": "user",
        "content": [{"type": "image_url", "image_url": {"url": url_3}}],
    }
]

outputs = llm.chat(messages=messages, sampling_params=sampling_params)
print(outputs[0].outputs[0].text)

In [ ]:
def generate_image_caption(image_path):
    """
    Génère une description textuelle (caption) pour une image donnée en utilisant un modèle de type BLIP.
    Args:
        image_path (str): Chemin vers l'image à analyser (doit être un fichier image lisible par PIL).
    Returns:
        str: Légende générée automatiquement décrivant le contenu visuel de l'image.
    """
    raw_image = Image.open(image_path).convert("RGB")
    inputs = processor(raw_image, return_tensors="pt")
    out = model.generate(**inputs, max_new_tokens=200)
    caption = processor.decode(out[0], skip_special_tokens=True)
    return caption

#### stockage dans un fichier csv

In [ ]:
def caption_dataset(img_paths, output_csv, img_folder="images"):
    """
    Génère un fichier CSV contenant les descriptions (captions) de toutes les images extraites.

    Args:
        img_paths (list): Liste des chemins relatifs des images (fichiers présents dans le dossier `img_folder`).
        output_csv (str): Chemin du fichier CSV à créer pour stocker les résultats.
        img_folder (str): Nom du dossier contenant les images (par défaut "images").

    Returns:
        pd.DataFrame: Un DataFrame contenant deux colonnes :
                      - 'image_filename': le nom du fichier image
                      - 'caption': la description générée automatiquement
    """
    data = []

    for path in img_paths:
        caption = generate_image_caption("images/"+path)
        data.append({
            "image_filename": os.path.basename(path),
            "caption": caption,
        })
        print(f" {os.path.basename(path)} → {img_paths.index(path)} → {caption}")

    df = pd.DataFrame(data)
    df.to_csv(output_csv, index=False, encoding="utf-8")
    print(f"\n CSV généré avec {len(df)} lignes :::: {output_csv}")
    return df

In [ ]:
img_paths=os.listdir("images/")

In [ ]:
caption_dataset(img_paths,"caption_image.csv")

 20240903-bnpparibas-csr-presentation-2024_page5_img1.png → 0 → a black and white image of a black and white image of a black and white image of a black and
